# Check-worthiness detection using Large Language Models

First, the necessary python modules are imported

In [2]:
%load_ext autoreload

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from claimbuster_utils import load_claimbuster_dataset
from tqdm.auto import tqdm
import json
import numpy as np
import re
import torch

/cluster/home/matssbra/.conda/envs/fakeNews/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-26 17:46:35.617300: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 17:46:35.742272: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 17:46:35.742307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02

## Load model and dataset

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    quantization_config = bnb_config,
    # attn_implementation="flash_attention_2", 
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    return_full_text=False,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

data = load_claimbuster_dataset("../data/ClaimBuster_Datasets/datasets")

Loading checkpoint shards: 100%|██████████| 3/3 [00:49<00:00, 16.38s/it]


## Zero-shot classification

In [4]:
with open("../prompts/ClaimBuster/standard/zero-shot.txt", "r") as f:
    instruction = f.read()

texts = data["Text"]
prompts = [f"{instruction} '''{text}'''" for text in texts]

class ProgressDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        return self.dataset[idx]
    def __len__(self):
        return len(self.dataset)
    
prompts_data = ProgressDataset(prompts)

dataset_with_scores = data.copy()

display(data.head())
dict_matcher = re.compile(r"{.*}")
score_matcher = re.compile(r"([Ss]core[^\d]*)(\d+)")


responses = pipe(prompts_data, batch_size=256)
for index, result in enumerate(tqdm(responses, total=len(prompts))):
    response = result[0]["generated_text"].replace('\n', "")
    dataset_index = data.index[index]
    try:
        parsed_json = json.loads(dict_matcher.search(r"{.*}", response).group(0))
        dataset_with_scores.loc[dataset_index, "score"] = parsed_json["score"]
        dataset_with_scores.loc[dataset_index, "reasoning"] = parsed_json["reasoning"]
    except (json.decoder.JSONDecodeError, AttributeError) as e:
        # Try to find score
        score = score_matcher.search(response)
        print(score)
        dataset_with_scores.loc[dataset_index, "score"] = score[2] if score else np.nan
        dataset_with_scores.loc[dataset_index, "reasoning"] = response
        continue
dataset_with_scores.to_csv("../results/ClaimBuster/zeroshot.csv", index=True)

,Verdict,Text
sentence_id,,
27247,1,We're 9 million jobs short of that.
10766,1,"You know, last year up to this time, we've los..."
3327,1,And in November of 1975 I was the first presid...
19700,1,And what we've done during the Bush administra...
12600,1,Do you know we don't have a single program spo...


  0%|          | 0/9674 [00:00<?, ?it/s]

  0%|          | 22/9674 [01:22<10:03:53,  3.75s/it]

0
90
0


AttributeError: 'NoneType' object has no attribute 'group'

## Discussion of results

In [ ]:
# Print the number of empty scores
print(dataset_with_scores[dataset_with_scores["score"].isna()])

             Verdict                                               Text  \
sentence_id                                                               
27247              1                We're 9 million jobs short of that.   
9453               1  But your rate comes down and the burden also c...   
2336               1  That's one of the unintended consequences of D...   
25315              1  Georgie Anne, we, believe me, supported the Si...   
20122              1  There are actually more people in government e...   
...              ...                                                ...   
15443              0  I'm sorry that we couldn't persuade the bringi...   
29611              0  It's because it's about time that this country...   
15531              0                           I made a good selection.   
16650              0  Well, in the uh - last two years, as I indicat...   
25557              0  Mr. Mondale, could I ask you to address the qu...   

             score      